In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

#import shap
from sklearn.inspection import permutation_importance
#from tqdm import tqdm
from sklearn.ensemble import IsolationForest
from datetime import datetime, timedelta

In [2]:
data_dir = "data/PdM/"

df_mst = pd.read_csv(data_dir + "ics_asset_mst.csv", na_values='\\N')
df_sigdata = pd.read_csv(data_dir + "55285839-9b78-48d8-9f4e-573190ace016_data.csv")
#df_sigdata = pd.read_csv(data_dir + "ics_asset_sigdata.csv", na_values='\\N')
df_status_hist = pd.read_csv(data_dir + "ics_asset_status_hist.csv", na_values='\\N')

df_mst.columns = ['ASSET_ID', 'ASSET_NAME', 'SENSOR_NUMBER']
#df_sigdata.columns = ['asset_id', 'created_at',	'temperature', 'voltage',
                      # 'rms_x', 'rms_y', 'rms_z', 'rms_xyz', 'vel_rms_x', 'vel_rms_y', 'vel_rms_z', 'vel_rms_xyz',
                      # 'skewness_x', 'skewness_y', 'skewness_z', 'vel_skewness_x', 'vel_skewness_y', 'vel_skewness_z',
                      # 'kurtosis_x', 'kurtosis_y', 'kurtosis_z', 'vel_kurtosis_x', 'vel_kurtosis_y', 'vel_kurtosis_z',
                      # 'crest_factor_x', 'crest_factor_y', 'crest_factor_z', 'vel_crest_factor_x', 'vel_crest_factor_y', 'vel_crest_factor_z',
                      # 'peak_x', 'peak_y', 'peak_z', 'vel_peak_x', 'vel_peak_y', 'vel_peak_z',
                      # 'peak2peak_x', 'peak2peak_y', 'peak2peak_z', 'vel_peak2peak_x', 'vel_peak2peak_y', 'vel_peak2peak_z']
df_status_hist.columns = ['asset_id', 'time', 'imbalance_health', 'misalignment_health', 'looseness_health', 'bearing_health', 'asset_health', 'CRT_DT']

In [3]:
df_sigdata = df_sigdata[df_sigdata['rms_xyz'] > 0.05].reset_index(drop=True)

In [4]:
# created_at을 datetime으로 변환
df_sigdata['created_at'] = pd.to_datetime(df_sigdata['created_at'], unit='s') + pd.Timedelta(hours=9)
# status_hist의 time을 datetime으로 변환
df_status_hist['time'] = pd.to_datetime(df_status_hist['time'])

In [5]:
# 1월 10일부터 7월 31일까지의 데이터만 사용
df_sigdata = df_sigdata[(df_sigdata['created_at'] >= '2024-01-10') & (df_sigdata['created_at'] <= '2024-07-31')].reset_index(drop=True)
df_status_hist = df_status_hist[(df_status_hist['time'] >= '2024-01-10') & (df_status_hist['time'] <= '2024-07-31')].reset_index(drop=True).drop('CRT_DT', axis=1)

In [6]:
# 전체 통계 정보를 저장할 리스트
modeling_list = []
proceed = 0

# 각 asset_id에 대해 데이터를 분리하고 모델 학습
for asset_id in df_sigdata['asset_id'].unique():
    proceed += 1
    print(f"Processing asset_id: {asset_id}  진행도: {proceed}/{len(df_sigdata['asset_id'].unique())}")
    
    # 해당 asset_id에 대한 데이터 필터링
    asset_data = df_sigdata[df_sigdata['asset_id'] == asset_id].copy()

    X = asset_data.drop(columns=['created_at', 'created_at_datetime', 'asset_id', 'temperature', 'time', 'imbalance_health', 'misalignment_health', 'looseness_health', 'bearing_health', 'asset_health'])

    # Isolation Forest 모델 적용
    model = IsolationForest(n_estimators=100, contamination=0.03, random_state=42)
    model.fit(X)
            
    # 이상값 탐지 결과 추가
    asset_data['anomaly'] = model.predict(X)

    # 모델링 결과 넣기
    modeling_list.append(asset_data)


Processing asset_id: 55285839-9b78-48d8-9f4e-573190ace016  진행도: 1/1


In [7]:
# 전체 결과를 하나의 데이터프레임으로 결합
final_result = pd.concat(modeling_list)

In [8]:
# 전체 결과 데이터
final_result.drop_duplicates(subset=['asset_id', 'created_at'], keep='first', inplace=True)
final_result

,asset_id,created_at,created_at_datetime,temperature,rms_x,rms_y,rms_z,rms_xyz,vel_rms_x,vel_rms_y,...,vel_peak2peak_x,vel_peak2peak_y,vel_peak2peak_z,time,imbalance_health,misalignment_health,looseness_health,bearing_health,asset_health,anomaly
0,55285839-9b78-48d8-9f4e-573190ace016,2024-01-10 02:47:53,2024-01-10 02:47:53,19.0,0.070012,0.069566,0.049347,0.110346,0.324538,0.358952,...,2.08901,2.44386,2.89197,2024-01-11,1.0,1.0,1.0,1.0,0.871241,1
1,55285839-9b78-48d8-9f4e-573190ace016,2024-01-10 08:48:07,2024-01-10 08:48:07,17.0,0.268893,0.226623,0.127837,0.374170,1.044890,1.030820,...,6.11164,6.91443,4.14951,2024-01-11,1.0,1.0,1.0,1.0,0.871241,1
2,55285839-9b78-48d8-9f4e-573190ace016,2024-01-10 10:48:16,2024-01-10 10:48:16,21.0,0.245302,0.245256,0.129588,0.370292,1.230610,1.010640,...,7.97459,6.41143,3.38071,2024-01-11,1.0,1.0,1.0,1.0,0.871241,1
3,55285839-9b78-48d8-9f4e-573190ace016,2024-01-10 14:48:28,2024-01-10 14:48:28,25.0,0.177727,0.208697,0.118564,0.298661,1.595570,1.599110,...,9.64796,9.38631,4.56256,2024-01-11,1.0,1.0,1.0,1.0,0.871241,1
4,55285839-9b78-48d8-9f4e-573190ace016,2024-01-10 16:48:31,2024-01-10 16:48:31,25.0,0.194262,0.218841,0.126226,0.318688,1.474350,1.511900,...,7.66562,8.19292,5.01201,2024-01-11,1.0,1.0,1.0,1.0,0.871241,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
854,55285839-9b78-48d8-9f4e-573190ace016,2024-07-28 09:16:42,2024-07-28 09:16:42,37.0,0.166773,0.227773,0.128130,0.310018,1.002450,0.831304,...,6.27584,5.87284,5.24028,2024-07-29,1.0,1.0,1.0,1.0,0.997219,1
855,55285839-9b78-48d8-9f4e-573190ace016,2024-07-29 09:17:50,2024-07-29 09:17:50,34.0,0.173119,0.227804,0.126015,0.312641,1.231730,0.765190,...,6.96598,4.78805,4.41407,2024-07-30,1.0,1.0,1.0,1.0,1.000000,1
856,55285839-9b78-48d8-9f4e-573190ace016,2024-07-29 13:18:01,2024-07-29 13:18:01,37.0,0.139693,0.195875,0.106624,0.263153,0.635817,0.489088,...,3.82300,3.40856,3.66680,2024-07-30,1.0,1.0,1.0,1.0,1.000000,1
857,55285839-9b78-48d8-9f4e-573190ace016,2024-07-29 15:18:04,2024-07-29 15:18:04,40.0,0.194491,0.228239,0.120181,0.323053,0.641991,0.654166,...,4.59598,4.51359,3.52848,2024-07-30,1.0,1.0,1.0,1.0,1.000000,1


In [9]:
# 비정상 데이터만
anomaly_df = final_result[final_result['anomaly'] == -1]
anomaly_df

,asset_id,created_at,created_at_datetime,temperature,rms_x,rms_y,rms_z,rms_xyz,vel_rms_x,vel_rms_y,...,vel_peak2peak_x,vel_peak2peak_y,vel_peak2peak_z,time,imbalance_health,misalignment_health,looseness_health,bearing_health,asset_health,anomaly
19,55285839-9b78-48d8-9f4e-573190ace016,2024-01-15 00:57:56,2024-01-15 00:57:56,28.0,0.261335,0.218215,0.122786,0.361926,3.943520,2.370380,...,20.85200,12.75380,7.57918,2024-01-16,1.000000,1.000000,1.000000,1.0,0.819881,-1
36,55285839-9b78-48d8-9f4e-573190ace016,2024-01-22 17:07:13,2024-01-22 17:07:13,19.0,0.032062,0.028804,0.027210,0.050971,0.385413,0.301386,...,3.63858,3.23386,4.69032,2024-01-23,1.000000,1.000000,1.000000,1.0,0.890369,-1
37,55285839-9b78-48d8-9f4e-573190ace016,2024-01-23 15:08:39,2024-01-23 15:08:39,18.0,0.166030,0.207260,0.126252,0.294045,1.574090,1.991870,...,9.00421,10.06030,3.93612,2024-01-24,0.939328,0.980000,0.980000,1.0,0.800932,-1
47,55285839-9b78-48d8-9f4e-573190ace016,2024-01-25 21:11:52,2024-01-25 21:11:52,19.0,0.234327,0.204773,0.113486,0.331240,2.505290,2.833960,...,13.27880,16.03900,6.22483,2024-01-26,1.000000,1.000000,1.000000,1.0,0.903824,-1
67,55285839-9b78-48d8-9f4e-573190ace016,2024-01-30 21:22:27,2024-01-30 21:22:27,24.0,0.212227,0.216375,0.117457,0.325045,2.675580,3.180920,...,14.33220,14.69870,7.47641,2024-01-31,1.000000,1.000000,1.000000,1.0,0.932022,-1
154,55285839-9b78-48d8-9f4e-573190ace016,2024-03-01 20:04:31,2024-03-01 20:04:31,20.0,0.043470,0.051086,0.048045,0.082509,0.361227,0.342286,...,3.12898,2.73394,3.49341,2024-03-02,1.000000,1.000000,1.000000,1.0,0.950000,-1
159,55285839-9b78-48d8-9f4e-573190ace016,2024-03-02 10:05:20,2024-03-02 10:05:20,12.0,0.030185,0.034134,0.029457,0.054258,0.372752,0.423200,...,4.20937,3.24049,2.52795,2024-03-03,1.000000,1.000000,1.000000,1.0,0.962304,-1
240,55285839-9b78-48d8-9f4e-573190ace016,2024-03-23 10:43:09,2024-03-23 10:43:09,31.0,0.226064,0.245130,0.151402,0.366219,3.787590,1.786490,...,19.22710,9.17719,7.66955,2024-03-24,1.000000,1.000000,1.000000,1.0,0.976886,-1
247,55285839-9b78-48d8-9f4e-573190ace016,2024-03-25 14:45:41,2024-03-25 14:45:41,27.0,0.034036,0.032808,0.025267,0.053602,0.404987,0.371243,...,3.75537,3.40508,4.51253,2024-03-26,1.000000,1.000000,1.000000,1.0,0.991188,-1
310,55285839-9b78-48d8-9f4e-573190ace016,2024-04-07 05:00:12,2024-04-07 05:00:12,25.0,0.164250,0.220735,0.118924,0.299741,2.876360,3.597620,...,15.85060,18.74450,6.88472,2024-04-08,1.000000,1.000000,1.000000,1.0,0.824773,-1


In [10]:
# final_data 전처리 후 csv로 확인

In [11]:
final_result.loc[:, 'time'] = pd.to_datetime(final_result['created_at']).dt.date

In [12]:
final_result['time'] = final_result['time'].astype('datetime64[ns]')

In [13]:
final_result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 859 entries, 0 to 858
Data columns (total 49 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   asset_id             859 non-null    object        
 1   created_at           859 non-null    datetime64[ns]
 2   created_at_datetime  859 non-null    object        
 3   temperature          859 non-null    float64       
 4   rms_x                859 non-null    float64       
 5   rms_y                859 non-null    float64       
 6   rms_z                859 non-null    float64       
 7   rms_xyz              859 non-null    float64       
 8   vel_rms_x            859 non-null    float64       
 9   vel_rms_y            859 non-null    float64       
 10  vel_rms_z            859 non-null    float64       
 11  vel_rms_xyz          859 non-null    float64       
 12  skewness_x           859 non-null    float64       
 13  skewness_y           859 non-null  

In [14]:
df_status_hist.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9541 entries, 0 to 9540
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   asset_id             9541 non-null   object        
 1   time                 9541 non-null   datetime64[ns]
 2   imbalance_health     6766 non-null   float64       
 3   misalignment_health  6766 non-null   float64       
 4   looseness_health     6766 non-null   float64       
 5   bearing_health       6766 non-null   float64       
 6   asset_health         6766 non-null   float64       
dtypes: datetime64[ns](1), float64(5), object(1)
memory usage: 521.9+ KB


In [15]:
final_result = pd.merge(
   final_result,
   df_status_hist[['asset_id', 'time', 'imbalance_health', 'misalignment_health', 'looseness_health', 'bearing_health', 'asset_health']],
   on=['asset_id', 'time'],
   how='left'
)

In [16]:
final_result.drop(columns=['time'], inplace=True)

In [17]:
print(final_result)

                                 asset_id          created_at  \
0    55285839-9b78-48d8-9f4e-573190ace016 2024-01-10 02:47:53   
1    55285839-9b78-48d8-9f4e-573190ace016 2024-01-10 08:48:07   
2    55285839-9b78-48d8-9f4e-573190ace016 2024-01-10 10:48:16   
3    55285839-9b78-48d8-9f4e-573190ace016 2024-01-10 14:48:28   
4    55285839-9b78-48d8-9f4e-573190ace016 2024-01-10 16:48:31   
..                                    ...                 ...   
854  55285839-9b78-48d8-9f4e-573190ace016 2024-07-28 09:16:42   
855  55285839-9b78-48d8-9f4e-573190ace016 2024-07-29 09:17:50   
856  55285839-9b78-48d8-9f4e-573190ace016 2024-07-29 13:18:01   
857  55285839-9b78-48d8-9f4e-573190ace016 2024-07-29 15:18:04   
858  55285839-9b78-48d8-9f4e-573190ace016 2024-07-29 17:18:14   

     created_at_datetime  temperature     rms_x     rms_y     rms_z   rms_xyz  \
0    2024-01-10 02:47:53         19.0  0.070012  0.069566  0.049347  0.110346   
1    2024-01-10 08:48:07         17.0  0.268893  0.226623

In [18]:
final_result.isna().sum()

asset_id                  0
created_at                0
created_at_datetime       0
temperature               0
rms_x                     0
rms_y                     0
rms_z                     0
rms_xyz                   0
vel_rms_x                 0
vel_rms_y                 0
vel_rms_z                 0
vel_rms_xyz               0
skewness_x                0
skewness_y                0
skewness_z                0
vel_skewness_x            0
vel_skewness_y            0
vel_skewness_z            0
kurtosis_x                0
kurtosis_y                0
kurtosis_z                0
vel_kurtosis_x            0
vel_kurtosis_y            0
vel_kurtosis_z            0
crest_factor_x            0
crest_factor_y            0
crest_factor_z            0
vel_crest_factor_x        0
vel_crest_factor_y        0
vel_crest_factor_z        0
peak_x                    0
peak_y                    0
peak_z                    0
vel_peak_x                0
vel_peak_y                0
vel_peak_z          

In [19]:
# csv파일로 저장
#final_result.to_csv('modeling_v02.csv', index=False)

In [20]:
columns_to_drop = [
    'temperature', 'rms_x', 'rms_y', 'rms_z', 'rms_xyz',
    'vel_rms_x', 'vel_rms_y', 'vel_rms_z', 'vel_rms_xyz',
    'skewness_x', 'skewness_y', 'skewness_z',
    'vel_skewness_x', 'vel_skewness_y', 'vel_skewness_z',
    'kurtosis_x', 'kurtosis_y', 'kurtosis_z',
    'vel_kurtosis_x', 'vel_kurtosis_y', 'vel_kurtosis_z',
    'crest_factor_x', 'crest_factor_y', 'crest_factor_z',
    'vel_crest_factor_x', 'vel_crest_factor_y', 'vel_crest_factor_z',
    'peak_x', 'peak_y', 'peak_z', 'vel_peak_x', 'vel_peak_y', 'vel_peak_z',
    'peak2peak_x', 'peak2peak_y', 'peak2peak_z',
    'vel_peak2peak_x', 'vel_peak2peak_y', 'vel_peak2peak_z'
]

In [21]:
final_result.drop(columns=columns_to_drop, inplace=True)

In [22]:
final_result

,asset_id,created_at,created_at_datetime,imbalance_health_x,misalignment_health_x,looseness_health_x,bearing_health_x,asset_health_x,anomaly,imbalance_health_y,misalignment_health_y,looseness_health_y,bearing_health_y,asset_health_y
0,55285839-9b78-48d8-9f4e-573190ace016,2024-01-10 02:47:53,2024-01-10 02:47:53,1.0,1.0,1.0,1.0,0.871241,1,0.924009,0.975,0.975,1.0,0.738013
1,55285839-9b78-48d8-9f4e-573190ace016,2024-01-10 08:48:07,2024-01-10 08:48:07,1.0,1.0,1.0,1.0,0.871241,1,0.924009,0.975,0.975,1.0,0.738013
2,55285839-9b78-48d8-9f4e-573190ace016,2024-01-10 10:48:16,2024-01-10 10:48:16,1.0,1.0,1.0,1.0,0.871241,1,0.924009,0.975,0.975,1.0,0.738013
3,55285839-9b78-48d8-9f4e-573190ace016,2024-01-10 14:48:28,2024-01-10 14:48:28,1.0,1.0,1.0,1.0,0.871241,1,0.924009,0.975,0.975,1.0,0.738013
4,55285839-9b78-48d8-9f4e-573190ace016,2024-01-10 16:48:31,2024-01-10 16:48:31,1.0,1.0,1.0,1.0,0.871241,1,0.924009,0.975,0.975,1.0,0.738013
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
854,55285839-9b78-48d8-9f4e-573190ace016,2024-07-28 09:16:42,2024-07-28 09:16:42,1.0,1.0,1.0,1.0,0.997219,1,1.000000,1.000,1.000,1.0,1.000000
855,55285839-9b78-48d8-9f4e-573190ace016,2024-07-29 09:17:50,2024-07-29 09:17:50,1.0,1.0,1.0,1.0,1.000000,1,1.000000,1.000,1.000,1.0,0.997219
856,55285839-9b78-48d8-9f4e-573190ace016,2024-07-29 13:18:01,2024-07-29 13:18:01,1.0,1.0,1.0,1.0,1.000000,1,1.000000,1.000,1.000,1.0,0.997219
857,55285839-9b78-48d8-9f4e-573190ace016,2024-07-29 15:18:04,2024-07-29 15:18:04,1.0,1.0,1.0,1.0,1.000000,1,1.000000,1.000,1.000,1.0,0.997219


In [23]:
# csv파일로 저장
#final_result.to_csv('modeling_status_v02.csv', index=False)

In [24]:
# 평가

In [25]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# 날짜별로 anomaly가 -1인 값이 하나라도 있는지 확인
final_result['date'] = final_result['created_at'].dt.date
anomaly_by_date = final_result.groupby(['asset_id', 'date'])['anomaly'].apply(lambda x: -1 in x.values).reset_index(name='is_anomaly')

# 날짜별로 asset_health가 1이 아닌 값이 있는지 확인
health_by_date = final_result.groupby(['asset_id', 'date'])['imbalance_health_x'].apply(lambda x: x.isna().any() or any(x != 1)).reset_index(name='is_not_healthy')

# 두 결과를 병합하여 평가 데이터셋 생성
evaluation_df = pd.merge(anomaly_by_date, health_by_date, on=['asset_id', 'date'])

# 실제 값과 예측 값
y_true = evaluation_df['is_not_healthy']
y_pred = evaluation_df['is_anomaly']

# 정확도, 정밀도, 재현율, F1 스코어 계산
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)

print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")


Accuracy: 0.81
Precision: 0.08
Recall: 0.13
F1 Score: 0.10


In [26]:
# True=비정상 데이터, False=정상 데이터
evaluation_df

,asset_id,date,is_anomaly,is_not_healthy
0,55285839-9b78-48d8-9f4e-573190ace016,2024-01-10,False,False
1,55285839-9b78-48d8-9f4e-573190ace016,2024-01-11,False,False
2,55285839-9b78-48d8-9f4e-573190ace016,2024-01-12,False,False
3,55285839-9b78-48d8-9f4e-573190ace016,2024-01-13,False,True
4,55285839-9b78-48d8-9f4e-573190ace016,2024-01-14,False,False
...,...,...,...,...
175,55285839-9b78-48d8-9f4e-573190ace016,2024-07-23,False,False
176,55285839-9b78-48d8-9f4e-573190ace016,2024-07-26,True,False
177,55285839-9b78-48d8-9f4e-573190ace016,2024-07-27,True,False
178,55285839-9b78-48d8-9f4e-573190ace016,2024-07-28,False,False


In [27]:
evaluation_df[evaluation_df['is_not_healthy'] == True]

,asset_id,date,is_anomaly,is_not_healthy
3,55285839-9b78-48d8-9f4e-573190ace016,2024-01-13,False,True
12,55285839-9b78-48d8-9f4e-573190ace016,2024-01-23,True,True
13,55285839-9b78-48d8-9f4e-573190ace016,2024-01-24,False,True
17,55285839-9b78-48d8-9f4e-573190ace016,2024-01-29,False,True
51,55285839-9b78-48d8-9f4e-573190ace016,2024-03-10,False,True
59,55285839-9b78-48d8-9f4e-573190ace016,2024-03-18,False,True
63,55285839-9b78-48d8-9f4e-573190ace016,2024-03-22,False,True
77,55285839-9b78-48d8-9f4e-573190ace016,2024-04-05,False,True
98,55285839-9b78-48d8-9f4e-573190ace016,2024-04-28,False,True
103,55285839-9b78-48d8-9f4e-573190ace016,2024-05-04,False,True


In [28]:
evaluation_df[evaluation_df['is_anomaly'] == True]

,asset_id,date,is_anomaly,is_not_healthy
5,55285839-9b78-48d8-9f4e-573190ace016,2024-01-15,True,False
11,55285839-9b78-48d8-9f4e-573190ace016,2024-01-22,True,False
12,55285839-9b78-48d8-9f4e-573190ace016,2024-01-23,True,True
14,55285839-9b78-48d8-9f4e-573190ace016,2024-01-25,True,False
18,55285839-9b78-48d8-9f4e-573190ace016,2024-01-30,True,False
43,55285839-9b78-48d8-9f4e-573190ace016,2024-03-01,True,False
44,55285839-9b78-48d8-9f4e-573190ace016,2024-03-02,True,False
64,55285839-9b78-48d8-9f4e-573190ace016,2024-03-23,True,False
66,55285839-9b78-48d8-9f4e-573190ace016,2024-03-25,True,False
79,55285839-9b78-48d8-9f4e-573190ace016,2024-04-07,True,False
